In [17]:
import numpy as np
import pandas as pd
from IPython.display import display

from tensorflow.keras.applications.efficientnet import EfficientNetB7,preprocess_input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [16]:
dataset_dir="../input/ranzcr-clip-catheter-line-classification/"

train=pd.read_csv(dataset_dir+"train.csv")
display(train)

,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,0,0,0,0,0,0,1,0,0,0,0,ec89415d1
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0,0,1,0,0,1,0,0,0,1,0,bf4c6da3c
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,0,0,0,0,0,0,0,0,1,0,0,3fc1c97e5
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0,0,0,0,0,0,0,1,0,0,0,c31019814
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0,0,0,0,0,0,0,0,0,1,0,207685cd1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30078,1.2.826.0.1.3680043.8.498.74257566841157531124...,0,0,1,0,0,0,0,0,1,1,0,5b5b9ac30
30079,1.2.826.0.1.3680043.8.498.46510939987173529969...,0,0,0,0,0,0,0,0,0,1,0,7192404d8
30080,1.2.826.0.1.3680043.8.498.43173270582850645437...,0,0,1,0,0,1,0,1,0,1,0,d4d1b066d
30081,1.2.826.0.1.3680043.8.498.95092491950130838685...,0,0,0,0,0,0,0,0,1,0,0,01a6602b8


In [11]:
efficientnet_b7=EfficientNetB7(include_top=False,weights="imagenet",pooling="avg")
efficientnet_b7.trainable=False
efficientnet_b7.summary()

______________________
block6k_drop (Dropout)          (None, None, None, 3 0           block6k_project_bn[0][0]         
__________________________________________________________________________________________________
block6k_add (Add)               (None, None, None, 3 0           block6k_drop[0][0]               
                                                                 block6j_add[0][0]                
__________________________________________________________________________________________________
block6l_expand_conv (Conv2D)    (None, None, None, 2 884736      block6k_add[0][0]                
__________________________________________________________________________________________________
block6l_expand_bn (BatchNormali (None, None, None, 2 9216        block6l_expand_conv[0][0]        
__________________________________________________________________________________________________
block6l_expand_activation (Acti (None, None, None, 2 0           block6l_expand_bn[0][

In [12]:
transfer_model=Sequential([
    efficientnet_b7,
    Dense(11,activation="sigmoid")
])
transfer_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 2560)              64097687  
_________________________________________________________________
dense_2 (Dense)              (None, 11)                28171     
Total params: 64,125,858
Trainable params: 28,171
Non-trainable params: 64,097,687
_________________________________________________________________


In [ ]:
adam=Adam(learning_rate=1e-4)
transfer_model.compile(optimizer=adam,loss="binary_crossentropy",metrics=["auc"])